Let us use Newton's method on the system defined by the function

In [1]:
function nlvalue(x)
    return [ exp(x[2]-x[1]) - 2,
          x[1]*x[2] + x[3],
          x[2]*x[3] + x[1]^2 - x[2]
        ]
end

nlvalue (generic function with 1 method)

Here is a function that computes the Jacobian matrix.

In [2]:
function nljac(x)
    J = zeros(3,3)
    J[1,:] = [-exp(x[2]-x[1]), exp(x[2]-x[1]), 0]
    J[2,:] = [x[2], x[1], 1]
    J[3,:] = [2*x[1], x[3]-1, x[2]]
    return J
end

nljac (generic function with 1 method)

(These functions could be written as separate files, or embedded within another function as we have done here.) Our initial guess at a root is the origin.

In [3]:
x = [0,0,0]

3-element Array{Int64,1}:
 0
 0
 0

We need the value (residual) of the nonlinear function, and its Jacobian, at this value for $\mathbf{x}$. 

In [4]:
@show f = nlvalue(x);
@show J = nljac(x);

f = nlvalue(x) = [-1.0, 0.0, 0.0]
J = nljac(x) = [-1.0 1.0 0.0; 0.0 0.0 1.0; 0.0 -1.0 0.0]


We solve for the Newton step and compute the new estimate. 

In [5]:
s = -(J\f)
x = [x x[:,1]+s]

3×2 Array{Float64,2}:
 0.0  -1.0
 0.0   0.0
 0.0   0.0

Here is the new residual.

In [6]:
nlvalue(x[:,2])

3-element Array{Float64,1}:
 0.7182818284590451
 0.0               
 1.0               

We don't seem to be especially close to a root. Let's iterate a few more times. 

In [7]:
for n = 2:7
    f = [f nlvalue(x[:,n])]
    s = -( nljac(x[:,n]) \ f[:,n] )
    x = [x x[:,n]+s]
end

We find the infinity norm of the residuals. 

In [8]:
residual_norm = maximum(abs.(f),dims=1)   # max in dimension 1

1×7 Array{Float64,2}:
 1.0  1.0  0.202293  0.0102521  2.15564e-5  1.98999e-10  1.38778e-17

We don't know an exact answer, so we will take the last computed value as its surrogate. 

In [9]:
r = x[:,end]
x = x[:,1:end-1]

3×7 Array{Float64,2}:
 0.0  -1.0  -0.578586  -0.463139  -0.458027  -0.458033  -0.458033
 0.0   0.0   0.157173   0.230904   0.235121   0.235114   0.235114
 0.0   0.0   0.157173   0.115452   0.107713   0.10769    0.10769 

The following will subtract r from every column of x.

In [10]:
e = x .- r

3×7 Array{Float64,2}:
  0.458033  -0.541967  -0.120553   -0.00510533  …  -1.07512e-10  0.0        
 -0.235114  -0.235114  -0.0779413  -0.00421021     -1.07432e-10  0.0        
 -0.10769   -0.10769    0.0494826   0.00776251      1.9768e-11   1.38778e-17

Now we take infinity norms and check for quadratic convergence. 

In [11]:
errs = maximum(abs.(e),dims=1)
ratios = @. log(errs[2:end]) / log(errs[1:end-1])

6-element Array{Float64,1}:
 0.7845032596064885
 3.4538622456644   
 2.296416733539853 
 2.196724973045181 
 2.1506706016502855
 1.6910878097784365

For a brief time, we see ratios around 2, but then the limitation of double precision makes it impossible for the doubling to continue. 